In [22]:
import pandas as pd
import numpy as np
import uuid
import random

In [2]:
df = pd.read_pickle('../data/1689_raw.pkl')

### Fix on Google Drive

In [4]:
df.to_excel('../data/temp_1689_for_google_drive.xlsx')

### Get File back from Google Drive

In [69]:
df = pd.read_excel('../data/temp_1689_for_google_drive_fixed.xlsx')
df['reference'] = df.reference.fillna(99).astype(int)
df['line'] = df.line.apply(lambda x: x.replace('Psalm ', 'Psalms ') if 'Psalm ' in x else x)
df['line'] = df.line.apply(lambda x: x.replace('Psalms ', 'Psalm ') if 'Psalms ' in x else x)
df = df.sort_values(by=['scripture', 'chapter', 'paragraph', 'reference']).drop(columns=['id']).rename(columns={'reference':'referenceNum'})
text = df[df.scripture==0].drop(columns=['scripture',]).rename(columns={'line':'paragraphText'})
refs = df[df.scripture==1].drop(columns=['scripture',]).rename(columns={'line':'paragraphRef'})

s = text.merge(refs, how='left', on=['chapter', 'paragraph', 'referenceNum'])
s['paragraphRef'] = s.paragraphRef.fillna("")
s['refArray'] = s.paragraphRef.apply(lambda x: [ref.strip() for ref in x.split(';') ])
s = s.drop(columns=['paragraphRef'])
s = s.rename(columns={'refArray':'paragraphRef'})
s = s.reset_index(drop=True)
s['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(s.shape[0])]
s['_type'] = 'confession'
s = s[['_id', '_type', 'chapter', 'paragraph', 'referenceNum', 'paragraphText', 'paragraphRef',]]

### Prep for Sanity schema

In [70]:
s.to_json("../cms/confession.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)